In [2]:
import pandas as pd

In [3]:
file_path = "/Users/sanyanema/Downloads/shiftdata.csv" # replace with local file path
df = pd.read_csv(file_path)

In [4]:
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266340 entries, 0 to 266339
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   SHIFT_ID          266340 non-null  object 
 1   WORKER_ID         266340 non-null  object 
 2   WORKPLACE_ID      266340 non-null  object 
 3   SHIFT_START_AT    266340 non-null  object 
 4   SHIFT_CREATED_AT  266340 non-null  object 
 5   OFFER_VIEWED_AT   266340 non-null  object 
 6   DURATION          266340 non-null  int64  
 7   SLOT              266340 non-null  object 
 8   CLAIMED_AT        13064 non-null   object 
 9   DELETED_AT        55644 non-null   object 
 10  IS_VERIFIED       266340 non-null  bool   
 11  CANCELED_AT       321 non-null     object 
 12  IS_NCNS           266340 non-null  bool   
 13  PAY_RATE          266340 non-null  float64
 14  CHARGE_RATE       266340 non-null  int64  
dtypes: bool(2), float64(1), int64(2), object(10)
memory usage: 26.9+ MB


(                   SHIFT_ID                 WORKER_ID  \
 0  6757580b1e2d97752fd69167  65b01f2e46c0645699081cbe   
 1  675d37d8a1ca6192a74d23f4  65298a18cc967a5cebbd40b6   
 2  67550bddd79613f860549322  6696d1c1d0200bf317ee5d3c   
 3  66f5d05de01fd3697b18c206  66b285d5d0200bf317738e59   
 4  66ee3848e62bb5f43e3baee5  620c6429e2ceb601ad203920   
 
                WORKPLACE_ID       SHIFT_START_AT     SHIFT_CREATED_AT  \
 0  5e7e45243bfbb200165914ae  2024-12-09 23:00:00  2024-12-09 20:50:19   
 1  5e1ce78827ff480016e9133e  2024-12-14 22:30:00   2024-12-14 7:46:32   
 2  626b0b89596c0601c2c39642  2024-12-08 15:00:00   2024-12-08 3:00:46   
 3  5cb9f07135163900163f532c  2024-09-27 14:00:00  2024-09-26 21:21:34   
 4  611af67795f4c501662edb31  2024-10-08 21:30:00   2024-09-21 3:06:48   
 
        OFFER_VIEWED_AT  DURATION SLOT CLAIMED_AT           DELETED_AT  \
 0  2024-12-09 21:18:42         8   pm        NaN                  NaN   
 1  2024-12-14 13:19:30         9   pm        NaN  2024-

In [5]:
# general overall stats 
summary_statistics = {
    "Total Shift Offers": len(df),
    "Total Unique Shifts": df["SHIFT_ID"].nunique(),
    "Claimed Shift Offers": df["CLAIMED_AT"].notnull().sum(),
    "Deleted Shift Offers": df["DELETED_AT"].notnull().sum(),
    "Deleted Shifts": df[df["DELETED_AT"].notnull()]["SHIFT_ID"].nunique(),
    "Claimed Shifts": df[df["CLAIMED_AT"].notnull()]["SHIFT_ID"].nunique(),
    "Claimed, not cancelled": df[(df["CLAIMED_AT"].notnull()) & (df["CANCELED_AT"].isnull())]["SHIFT_ID"].nunique(),
    "Canceled Shift Offers": df["CANCELED_AT"].notnull().sum(),
    "Unclaimed Shift Offers": df["CLAIMED_AT"].isna().sum(),
    "Average Pay Rate": df["PAY_RATE"].mean(),
    "Average Charge Rate": df["CHARGE_RATE"].mean(),
    "Average Pay Rate for Claimed Shift Offers": df[df["CLAIMED_AT"].notnull()]["PAY_RATE"].mean(),
    "Average Pay Rate for Deleted Shift Offers": df[df["DELETED_AT"].notnull()]["PAY_RATE"].mean(), 
    "Average Pay Rate for Unclaimed Shift Offers": df[df["CLAIMED_AT"].isnull()]["PAY_RATE"].mean(),
    "Total Workers": df["WORKER_ID"].nunique(),
    "Total Workplaces": df["WORKPLACE_ID"].nunique()
}

summary_statistics

{'Total Shift Offers': 266340,
 'Total Unique Shifts': 19900,
 'Claimed Shift Offers': np.int64(13064),
 'Deleted Shift Offers': np.int64(55644),
 'Deleted Shifts': 3671,
 'Claimed Shifts': 12802,
 'Claimed, not cancelled': 12626,
 'Canceled Shift Offers': np.int64(321),
 'Unclaimed Shift Offers': np.int64(253276),
 'Average Pay Rate': np.float64(24.164935984080497),
 'Average Charge Rate': np.float64(31.511905834647443),
 'Average Pay Rate for Claimed Shift Offers': np.float64(26.559910440906307),
 'Average Pay Rate for Deleted Shift Offers': np.float64(23.212351376608442),
 'Average Pay Rate for Unclaimed Shift Offers': np.float64(24.04140297541023),
 'Total Workers': 10291,
 'Total Workplaces': 132}

In [8]:
claimed_shifts = df[df["CLAIMED_AT"].notnull()].groupby("SHIFT_ID").size().reset_index(name="TOTAL_CLAIMS")
total_shifts = df.groupby("SHIFT_ID").size().reset_index(name="TOTAL_OFFERS")

claim_rate_per_shift = total_shifts.merge(claimed_shifts, on="SHIFT_ID", how="left").fillna(0)
claim_rate_per_shift["CLAIM_RATE"] = claim_rate_per_shift["TOTAL_CLAIMS"] / claim_rate_per_shift["TOTAL_OFFERS"] 
claim_rate_per_shift[claim_rate_per_shift["CLAIM_RATE"] == 0].sort_values(by="TOTAL_OFFERS", ascending=False) # shifts that get no claims but are posted a lot - lots of offers for these shifts
claim_rate_per_shift[claim_rate_per_shift["TOTAL_CLAIMS"] > 0].sort_values(by="TOTAL_OFFERS", ascending=False) 
claim_rate_per_shift[claim_rate_per_shift["TOTAL_CLAIMS"] > 1].sort_values(by="TOTAL_OFFERS", ascending=False) # this probably means one of the other people cancelled or didn't work the shift, so someone else went. total_claims > 1

# why is there discrepancy between canceled, verified, and ncns - is the tracking for this manual?
# some of them have verified status of true for all x amount of workers - are these special types of shifts or is something wrong here

,SHIFT_ID,TOTAL_OFFERS,TOTAL_CLAIMS,CLAIM_RATE
13782,6754b23e36deda6cc29e55f1,219,1.0,0.004566
6846,6716d77eeaf203a9be830901,218,1.0,0.004587
5510,670bd67a642c9c40efa05c65,204,1.0,0.004902
17002,6770c8845b9214031e3dfe69,194,1.0,0.005155
8283,671fd42a2b12f49dca9b946e,185,1.0,0.005405
...,...,...,...,...
6460,67142c8cb28d377e529048a5,1,1.0,1.000000
6461,67142de2b28d377e52905f97,1,1.0,1.000000
13565,67523967cd245fc6b09b87a3,1,1.0,1.000000
6462,67142f51393da443b763fa79,1,1.0,1.000000


In [10]:
# extract the number of shifts that have a charge rate < pay rate and all info associated
viable_shifts = df[(df["CLAIMED_AT"].notnull()) & (df["CHARGE_RATE"] < df["PAY_RATE"])]["SHIFT_ID"].nunique() #claimed
vshifts = df[(df["CHARGE_RATE"] < df["PAY_RATE"])].nunique() #all, not necessarily claimed

viable_shifts

2356

In [20]:
# convert datetime columns to proper format
df["SHIFT_START_AT"] = pd.to_datetime(df["SHIFT_START_AT"])
df["SHIFT_CREATED_AT"] = pd.to_datetime(df["SHIFT_CREATED_AT"])
df["OFFER_VIEWED_AT"] = pd.to_datetime(df["OFFER_VIEWED_AT"])
df["CLAIMED_AT"] = pd.to_datetime(df["CLAIMED_AT"])
df["DELETED_AT"] = pd.to_datetime(df["DELETED_AT"])
df["CANCELED_AT"] = pd.to_datetime(df["CANCELED_AT"])

# extract hour of shift start for time-based analysis
df["SHIFT_HOUR"] = df["SHIFT_START_AT"].dt.hour

# need to agregate by shift_id otherwise you will count multiple offers as shifts which is wrong
time_analysis = df.groupby("SHIFT_HOUR").agg(
    Total_Shifts=("SHIFT_ID", "nunique"),  # Unique shifts, not offers
    Claimed_Shifts=("SHIFT_ID", lambda x: x[df["CLAIMED_AT"].notna()].nunique()),
    Deleted_Shifts=("SHIFT_ID", lambda x: x[df["DELETED_AT"].notna()].nunique()),
).reset_index()

# Add claim and delete rates
time_analysis["Claim_Rate"] = time_analysis["Claimed_Shifts"] / time_analysis["Total_Shifts"]
time_analysis["Delete_Rate"] = time_analysis["Deleted_Shifts"] / time_analysis["Total_Shifts"]

# Handle division by zero (if Total_Shifts is 0)
time_analysis.fillna(0, inplace=True)
time_analysis

time_analysis.sort_values(by=[ "Total_Shifts", "Claim_Rate"], ascending=[ False, False]) # earlier shifts first, then higher claim rates,then higher amt of shifts
# sort by claim rate but also high volume of total shifts. 100% claim rate is not that interesting if there is only 1 total shift

# there are 266k offers, but only one is posted at 10am - so interesting

time_analysis.sort_values(by=["SHIFT_HOUR"]) # chronological

# average claim rate across the hours
time_analysis["Claim_Rate"].mean()


np.float64(0.6025038538630102)

In [13]:
# Extract hour of shift start for time-based analysis for SHIFT OFFERS
df["SHIFT_HOUR"] = df["SHIFT_START_AT"].dt.hour

# need to agregate by shift_id otherwise you will count multiple offers as shifts which is wrong
time_analysis = df.groupby("SHIFT_HOUR").agg(
    Total_Shift_Offers=("SHIFT_ID", "count"),  # shift offers
    Claimed_Shift_Offers=("CLAIMED_AT", "count"),
    Deleted_Shift_Offers=("DELETED_AT", "count"),
    Viewed_Shift_Offers=("OFFER_VIEWED_AT", "count")
).reset_index()

time_analysis

,SHIFT_HOUR,Total_Shift_Offers,Claimed_Shift_Offers,Deleted_Shift_Offers,Viewed_Shift_Offers
0,0,3116,164,714,3116
1,1,4290,437,1222,4290
2,2,797,40,144,797
3,3,268,21,19,268
4,4,197,6,7,197
5,5,16614,972,4197,16614
6,6,29269,1646,4437,29269
7,7,7601,470,1041,7601
8,8,57,8,10,57
9,9,204,5,23,204


In [248]:
# Analyze shift claim and deletion rates by workplace
workplace_analysis = df.groupby("WORKPLACE_ID").agg(
    Claimed_Shifts=("SHIFT_ID", lambda x: x[df["CLAIMED_AT"].notna()].nunique()),
    Deleted_Shifts=("SHIFT_ID", lambda x: x[df["DELETED_AT"].notna()].nunique()),
    Total_Shifts=("SHIFT_ID", "nunique"),
).reset_index()

workplace_shift_offer_analysis = df.groupby("WORKPLACE_ID").agg(
    Claimed_Shifts=("CLAIMED_AT", "count"),
    Deleted_Shifts=("DELETED_AT", "count"),
    Total_Shifts=("SHIFT_ID", "count"),
).reset_index()

# Calculate claim and delete percentages
workplace_analysis["Claim_Rate"] = workplace_analysis["Claimed_Shifts"] / workplace_analysis["Total_Shifts"] * 100
workplace_analysis["Delete_Rate"] = workplace_analysis["Deleted_Shifts"] / workplace_analysis["Total_Shifts"] * 100

# Identify workplaces with extreme claim or delete rates
high_claim_workplaces = workplace_analysis.sort_values(by="Claim_Rate", ascending=False).head(10) # whose shifts gets claimed the most
high_delete_workplaces = workplace_analysis.sort_values(by="Delete_Rate", ascending=False).head(10) # who deletes the most
high_shifts = workplace_analysis.sort_values(by="Total_Shifts", ascending=False).head(10) # who has the most shifts

workplace_analysis["Total_Shifts"].sum()
workplace_shift_offer_analysis["Total_Shifts"].mean() # this is for shift offers, not unique shifts
workplace_shift_offer_analysis["Deleted_Shifts"].mean() # mean of shift offer deletion rates

,WORKPLACE_ID,Claimed_Shifts,Deleted_Shifts,Total_Shifts,Claim_Rate,Delete_Rate
55,611af67795f4c501662edb31,956,505,1779,53.738055,28.386734
1,5bdb65eb27415b0004330ace,754,330,1220,61.803279,27.049180
21,5ebf09a7fe8b200017aeb9eb,539,177,949,56.796628,18.651212
18,5e95d5f5cf5e8d001653314e,478,178,832,57.451923,21.394231
22,5ebf16f8fe8b200017aebe0f,391,170,752,51.994681,22.606383
119,65428c5eb9ae7bfe06a31fec,445,125,682,65.249267,18.328446
66,61a568e0cb11990185bb97c7,369,129,620,59.516129,20.806452
97,637e71fd4a702e01b5e6261b,333,142,592,56.250000,23.986486
24,5ebf1773a253570017a27da2,495,42,582,85.051546,7.216495
53,610c3e4bb0d8850166b2bd41,479,22,552,86.775362,3.985507


In [26]:
# lead time analysis

df["SHIFT_CREATED_AT"] = pd.to_datetime(df["SHIFT_CREATED_AT"])
df["SHIFT_START_AT"] = pd.to_datetime(df["SHIFT_START_AT"])

df["LEAD_TIME"] = (df["SHIFT_START_AT"] - df["SHIFT_CREATED_AT"])

lead_time_per_shift = df.groupby("SHIFT_ID")["LEAD_TIME"].mean().reset_index()

lead_time_analysis = df.groupby("SHIFT_ID").agg(
    LEAD_TIME=("LEAD_TIME", "mean"),  # Correct mean calculation
    CLAIM_COUNT=("CLAIMED_AT", "count")  # Count number of claims per shift
).reset_index()

untimely_shifts = lead_time_analysis[lead_time_analysis["LEAD_TIME"] < pd.Timedelta(hours=1)] # 1120 shifts
claimed_untimely_shifts = untimely_shifts[untimely_shifts["CLAIM_COUNT"] > 0] # 483 shifts claimed
claimed_untimely_shifts
lead_time_analysis["LEAD_TIME"].mean() # mean lead time

# shifts with negative lead times - the shift is posted after the shift starts, likely some error in the data
invalid_shifts = lead_time_per_shift[lead_time_per_shift["LEAD_TIME"] < pd.Timedelta(0)]


1120


,SHIFT_ID,LEAD_TIME
2155,66f6d1bf6ea5c51c6f939138,-2 days +22:05:17
2158,66f6d27b55acbba28a5fd498,-1 days +13:47:08
3034,66fc0073edb1b00f4731a978,-1 days +01:19:40
3036,66fc01facf2a0ef62d261336,-1 days +23:08:09
3965,66fff0aa4e8bd2cfeba41736,-1 days +22:17:57
3966,66fff0ac55a904babd63f1b0,-1 days +22:17:56
4341,6703f3fe8ea7ea7d7a967692,-4 days +22:29:41
4848,6706f02b0fbcd27eb812eee4,-18 days +15:24:13
4855,6706f2ce0fbcd27eb8131398,-1 days +14:32:58
5172,67092d35bab523ddda9089d5,-1 days +15:39:15


In [55]:
df["TIME_TO_DELETE"] = (df["SHIFT_START_AT"] - df["DELETED_AT"]) # what is the time between a shift starting and getting deleted, if deleted
overall_delete_time = df["TIME_TO_DELETE"].mean() # on average 
delete_time_per_shift = df.groupby("SHIFT_ID")["TIME_TO_DELETE"].mean().reset_index() # per shift
delete_per_shift = delete_time_per_shift["TIME_TO_DELETE"].mean() 

overall_delete_time, delete_per_shift

(Timedelta('1 days 21:01:29.781845302'),
 Timedelta('3 days 00:44:26.695178425'))

In [46]:
duration_analysis = df.groupby("SHIFT_ID").agg(
    DURATION=("DURATION", "mean"),  
    CLAIM_COUNT=("CLAIMED_AT", "count")  # Count number of claims per shift
).reset_index()

duration_analysis[duration_analysis["CLAIM_COUNT"] == 0] # which shifts don't get claims
duration_analysis[(duration_analysis["DURATION"] > 9) & (duration_analysis["CLAIM_COUNT"] == 0)] # how many of unclaimed shifts are of duration > 9 hrs

,SHIFT_ID,DURATION,CLAIM_COUNT
13,66c61628d49f76fef903e7f9,12.0,0
28,66c64856217c10464cdce702,12.0,0
30,66c64859217c10464cdce77e,12.0,0
31,66c64889d01077b2f878f200,12.0,0
33,66c64a37d01077b2f8791f3b,12.0,0
...,...,...,...
19793,678c5297407aea5677e94d06,11.0,0
19797,678c681b85ba290bdec83260,10.0,0
19819,678ca875407aea5677ed64d8,12.0,0
19825,678cf7b4c59905eff8b8e793,12.0,0


In [155]:
# THIS IS ABOUT SHIFT OFFERS NOT UNIQUE SHIFTS

untimely_shifts = df[df["LEAD_TIME"] < pd.Timedelta(hours=1)] # 9174 shifts
deleted = untimely_shifts[untimely_shifts["DELETED_AT"].notna()] # 1025 shifts - this is good practice
claimed_untimely_shifts = untimely_shifts[untimely_shifts["CLAIMED_AT"].notna()] # 490 shifts
unclaimed_untimely_shifts = untimely_shifts[untimely_shifts["CLAIMED_AT"].isnull()] # 8684 shifts
untimely_shifts

,SHIFT_ID,WORKER_ID,WORKPLACE_ID,SHIFT_START_AT,SHIFT_CREATED_AT,OFFER_VIEWED_AT,DURATION,SLOT,CLAIMED_AT,DELETED_AT,IS_VERIFIED,CANCELED_AT,IS_NCNS,PAY_RATE,CHARGE_RATE,SHIFT_HOUR,time,LEAD_TIME
38,66fda9938660269a71681b79,66b3fd8280985b3f55242852,610c3e4bb0d8850166b2bd41,2024-10-02 21:00:00,2024-10-02 20:14:11,2024-10-02 20:25:43,8,pm,NaT,NaT,False,NaT,False,24.34,31,21,0 days 00:45:49,0 days 00:45:49
70,670a82e0642c9c40ef9199d2,6640551b5859f42389c8c515,5ebf16f8fe8b200017aebe0f,2024-10-12 14:30:00,2024-10-12 14:08:32,2024-10-12 14:18:00,8,am,NaT,NaT,False,NaT,False,26.74,26,14,0 days 00:21:28,0 days 00:21:28
90,678d8221bfb372556054dbc8,64c625e9ffacc00ad3abe069,5fffa958539a730016772b5c,2025-01-19 23:00:00,2025-01-19 22:52:17,2025-01-20 00:06:47,7,pm,NaT,NaT,False,NaT,False,31.99,26,23,0 days 00:07:43,0 days 00:07:43
120,670d297cf61fcbf16d84f0c7,6200babfe574d701aa260ee5,615b663fbd4bd10188739177,2024-10-14 15:00:00,2024-10-14 14:23:56,2024-10-14 14:56:20,8,am,NaT,2024-10-14 14:57:42,False,NaT,False,22.53,43,15,0 days 00:36:04,0 days 00:36:04
122,6774939264d37ece4fc0490b,6325cfcd9603d78083882405,61b0d0ed0a1f9a01805c0531,2025-01-01 02:00:00,2025-01-01 01:00:02,2025-01-01 01:07:06,12,noc,2025-01-01 01:07:20,NaT,True,NaT,False,48.43,40,2,0 days 00:59:58,0 days 00:59:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266151,6758502a04dba87f4c64477c,660a69546b5cf282194bfcd2,5e95d5f5cf5e8d001653314e,2024-12-10 15:00:00,2024-12-10 14:28:59,2024-12-10 14:56:09,8,am,NaT,NaT,False,NaT,False,23.19,37,15,0 days 00:31:01,0 days 00:31:01
266166,6735266c6fc08d00ef2cb162,66be4a64ddc24e527263d974,5e95d5f5cf5e8d001653314e,2024-11-13 23:00:00,2024-11-13 22:21:32,2024-11-13 22:32:53,8,pm,NaT,2024-11-13 22:58:13,False,NaT,False,22.34,37,23,0 days 00:38:28,0 days 00:38:28
266253,6723f9aaee4e6e38920ad905,66945b63d0200bf3177f16f4,5f52a9138c405c0016d30feb,2024-10-31 22:00:00,2024-10-31 21:42:03,2024-10-31 21:50:26,7,pm,NaT,NaT,False,NaT,False,21.81,29,22,0 days 00:17:57,0 days 00:17:57
266268,66fda9938660269a71681b80,66f4e6cef472d33b6812de57,610c3e4bb0d8850166b2bd41,2024-10-02 21:00:00,2024-10-02 20:14:11,2024-10-02 20:25:40,8,pm,NaT,NaT,False,NaT,False,24.96,31,21,0 days 00:45:49,0 days 00:45:49


In [239]:
# Group by worker ID and count the number of shifts they viewed
worker_views = df.groupby("WORKER_ID")["SHIFT_ID"].nunique().reset_index()
worker_views.rename(columns={"SHIFT_ID": "TOTAL_VIEWS"}, inplace=True)

# Count the number of shifts each worker actually claimed
worker_claims = df[df["CLAIMED_AT"].notna()].groupby("WORKER_ID")["SHIFT_ID"].nunique().reset_index()
worker_claims.rename(columns={"SHIFT_ID": "TOTAL_CLAIMS"}, inplace=True)

# Merge both datasets to compare
worker_behavior = worker_views.merge(worker_claims, on="WORKER_ID", how="left").fillna(0)

# Calculate claim rate per worker
worker_behavior["CLAIM_RATE"] = worker_behavior["TOTAL_CLAIMS"] / worker_behavior["TOTAL_VIEWS"]

worker_behavior_sorted = worker_behavior.sort_values(by=["CLAIM_RATE","TOTAL_VIEWS"], ascending=[False,False]) # these are the top workers that have pretty good claim rates! could indicate an experienced worker 

# Select three workers for deep dive
top_claimer = worker_behavior_sorted.iloc[0]  # High claim rate
low_claimer = worker_behavior_sorted[worker_behavior_sorted["CLAIM_RATE"] > 0].iloc[-1]  # Low claim rate but still claims
mid_claimer = worker_behavior_sorted.iloc[len(worker_behavior_sorted) // 2]  # Middle ground

# Get their IDs
selected_workers = [top_claimer["WORKER_ID"], low_claimer["WORKER_ID"], mid_claimer["WORKER_ID"]]

# Filter the main dataset for only those workers
worker_journey = df[df["WORKER_ID"].isin(selected_workers)] # filter the df for the workers found above


['654ae8657ff7c6eee5adffb4',
 '66f356e7f472d33b68a0a20c',
 '635eb1b37ccb0c3b06f621c7']

In [253]:
# how quick are shifts getting claimed
df["TIME_TO_CLAIM"] = df["CLAIMED_AT"] - df["SHIFT_CREATED_AT"]
time_to_claim_analysis = df.groupby("WORKER_ID")["TIME_TO_CLAIM"].mean().reset_index()
time_to_claim_analysis.sort_values(by="TIME_TO_CLAIM").head(50)

,WORKER_ID,TIME_TO_CLAIM
326,60a336ae19d2770166f25add,0 days 00:00:00
4827,646177ea4107b99bc1305781,0 days 00:00:00
9686,673bb8aa85353a6222952903,0 days 00:00:00
8238,66a5f99ed0200bf3171e32c1,0 days 00:00:00
4805,645d2c134107b99bc118ddf0,0 days 00:00:00
5195,649f4f27deef5579a85e270d,0 days 00:00:00
7188,661c5268b44011b92c9c033d,0 days 00:00:00
8459,66bf640c93d8c271682ff722,0 days 00:00:00.166666666
2163,626874fbc5b1ba01b7437fc2,0 days 00:00:00.250000
5688,64fb78acbe202a3a2e41cdfe,0 days 00:00:00.333333333
